# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [48]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values = 'NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values = 'NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values = 'NA', infer_schema_length=1000)

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-534664358.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [49]:
# Write your SQL query here
result = ctx.execute("""
  SELECT DISTINCT carrier, name
  FROM airlines
  ORDER BY carrier
""")

print(result)

shape: (16, 2)
┌─────────┬────────────────────────┐
│ carrier ┆ name                   │
│ ---     ┆ ---                    │
│ str     ┆ str                    │
╞═════════╪════════════════════════╡
│ 9E      ┆ Endeavor Air Inc.      │
│ AA      ┆ American Airlines Inc. │
│ AS      ┆ Alaska Airlines Inc.   │
│ B6      ┆ JetBlue Airways        │
│ DL      ┆ Delta Air Lines Inc.   │
│ …       ┆ …                      │
│ UA      ┆ United Air Lines Inc.  │
│ US      ┆ US Airways Inc.        │
│ VX      ┆ Virgin America         │
│ WN      ┆ Southwest Airlines Co. │
│ YV      ┆ Mesa Airlines Inc.     │
└─────────┴────────────────────────┘


### 1.2 Find the top 10 destinations by number of flights

In [50]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    f.dest,
    a.name AS destination_name,
    COUNT(*) AS num_flights
  FROM flights AS f
  LEFT JOIN airports AS a
    ON f.dest = a.faa
  GROUP BY f.dest, a.name
  ORDER BY num_flights DESC
  LIMIT 10
""")

print(result)

shape: (10, 3)
┌──────┬─────────────────────────────────┬─────────────┐
│ dest ┆ destination_name                ┆ num_flights │
│ ---  ┆ ---                             ┆ ---         │
│ str  ┆ str                             ┆ u32         │
╞══════╪═════════════════════════════════╪═════════════╡
│ ORD  ┆ Chicago Ohare Intl              ┆ 17283       │
│ ATL  ┆ Hartsfield Jackson Atlanta Int… ┆ 17215       │
│ LAX  ┆ Los Angeles Intl                ┆ 16174       │
│ BOS  ┆ General Edward Lawrence Logan … ┆ 15508       │
│ MCO  ┆ Orlando Intl                    ┆ 14082       │
│ CLT  ┆ Charlotte Douglas Intl          ┆ 14064       │
│ SFO  ┆ San Francisco Intl              ┆ 13331       │
│ FLL  ┆ Fort Lauderdale Hollywood Intl  ┆ 12055       │
│ MIA  ┆ Miami Intl                      ┆ 11728       │
│ DCA  ┆ Ronald Reagan Washington Natl   ┆ 9705        │
└──────┴─────────────────────────────────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [51]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    year, month, day, carrier, flight, origin, dest, dep_time, dep_delay
  FROM flights
  WHERE dep_delay > 120
  ORDER BY dep_delay DESC
""")

print(result)

shape: (9_723, 9)
┌──────┬───────┬─────┬─────────┬───┬────────┬──────┬──────────┬───────────┐
│ year ┆ month ┆ day ┆ carrier ┆ … ┆ origin ┆ dest ┆ dep_time ┆ dep_delay │
│ ---  ┆ ---   ┆ --- ┆ ---     ┆   ┆ ---    ┆ ---  ┆ ---      ┆ ---       │
│ i64  ┆ i64   ┆ i64 ┆ str     ┆   ┆ str    ┆ str  ┆ i64      ┆ i64       │
╞══════╪═══════╪═════╪═════════╪═══╪════════╪══════╪══════════╪═══════════╡
│ 2013 ┆ 1     ┆ 9   ┆ HA      ┆ … ┆ JFK    ┆ HNL  ┆ 641      ┆ 1301      │
│ 2013 ┆ 6     ┆ 15  ┆ MQ      ┆ … ┆ JFK    ┆ CMH  ┆ 1432     ┆ 1137      │
│ 2013 ┆ 1     ┆ 10  ┆ MQ      ┆ … ┆ EWR    ┆ ORD  ┆ 1121     ┆ 1126      │
│ 2013 ┆ 9     ┆ 20  ┆ AA      ┆ … ┆ JFK    ┆ SFO  ┆ 1139     ┆ 1014      │
│ 2013 ┆ 7     ┆ 22  ┆ MQ      ┆ … ┆ JFK    ┆ CVG  ┆ 845      ┆ 1005      │
│ …    ┆ …     ┆ …   ┆ …       ┆ … ┆ …      ┆ …    ┆ …        ┆ …         │
│ 2013 ┆ 9     ┆ 2   ┆ B6      ┆ … ┆ JFK    ┆ SJU  ┆ 2302     ┆ 121       │
│ 2013 ┆ 9     ┆ 11  ┆ EV      ┆ … ┆ EWR    ┆ SAV  ┆ 1850     ┆ 121   

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [52]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    f.origin,
    a.name AS airport_name,
    ROUND(AVG(f.dep_delay), 2) AS avg_dep_delay,
    COUNT(*) AS num_flights
  FROM flights AS f
  LEFT JOIN airports AS a
    ON f.origin = a.faa
  GROUP BY f.origin, a.name
  ORDER BY avg_dep_delay DESC
""")

print(result)

shape: (3, 4)
┌────────┬─────────────────────┬───────────────┬─────────────┐
│ origin ┆ airport_name        ┆ avg_dep_delay ┆ num_flights │
│ ---    ┆ ---                 ┆ ---           ┆ ---         │
│ str    ┆ str                 ┆ f64           ┆ u32         │
╞════════╪═════════════════════╪═══════════════╪═════════════╡
│ EWR    ┆ Newark Liberty Intl ┆ 15.11         ┆ 120835      │
│ JFK    ┆ John F Kennedy Intl ┆ 12.11         ┆ 111279      │
│ LGA    ┆ La Guardia          ┆ 10.35         ┆ 104662      │
└────────┴─────────────────────┴───────────────┴─────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [53]:
# First, let's check what columns are available
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
print(result)

# Now write your query to find busiest month
result = ctx.execute("""
  SELECT
    month,
    COUNT(*) AS num_flights
  FROM flights
  GROUP BY month
  ORDER BY num_flights DESC
  LIMIT 1
""")

print(result)

shape: (5, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1400     ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1416     ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1089     ┆ 5    ┆ 40     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 762      ┆ 6    ┆ 0      ┆ 2013-01-01 11:00:00 UTC │
└──────┴───────┴─────┴──────────┴───┴──────────┴──────┴────────┴───────────

### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [54]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    f.carrier,
    a.name AS airline,
    ROUND(AVG(CASE WHEN f.dep_delay <= 15 THEN 1.0 ELSE 0.0 END) * 100, 2) AS on_time_pct,
    COUNT(*) AS num_flights
  FROM flights AS f
  JOIN airlines AS a
    ON f.carrier = a.carrier
  GROUP BY f.carrier, a.name
  ORDER BY on_time_pct DESC
""")

print(result)

shape: (16, 4)
┌─────────┬─────────────────────────────┬─────────────┬─────────────┐
│ carrier ┆ airline                     ┆ on_time_pct ┆ num_flights │
│ ---     ┆ ---                         ┆ ---         ┆ ---         │
│ str     ┆ str                         ┆ f64         ┆ u32         │
╞═════════╪═════════════════════════════╪═════════════╪═════════════╡
│ HA      ┆ Hawaiian Airlines Inc.      ┆ 92.98       ┆ 342         │
│ AS      ┆ Alaska Airlines Inc.        ┆ 86.55       ┆ 714         │
│ US      ┆ US Airways Inc.             ┆ 84.99       ┆ 20536       │
│ DL      ┆ Delta Air Lines Inc.        ┆ 83.07       ┆ 48110       │
│ AA      ┆ American Airlines Inc.      ┆ 82.44       ┆ 32729       │
│ …       ┆ …                           ┆ …           ┆ …           │
│ FL      ┆ AirTran Airways Corporation ┆ 71.69       ┆ 3260        │
│ F9      ┆ Frontier Airlines Inc.      ┆ 71.53       ┆ 685         │
│ 9E      ┆ Endeavor Air Inc.           ┆ 70.12       ┆ 18460       │
│ EV 

## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [55]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    f.carrier,
    a.name AS airline,
    f.flight,
    f.origin,
    f.dest
  FROM flights AS f
  JOIN airlines AS a
    ON f.carrier = a.carrier
  ORDER BY f.year, f.month, f.day, f.dep_time
  LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline                ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [56]:
# Write your SQL query here
result = ctx.execute("""
    WITH fleet AS (
    SELECT DISTINCT
      f.carrier,
      f.tailnum,
      p.year
    FROM flights AS f
    JOIN planes  AS p
      ON f.tailnum = p.tailnum
    WHERE p.year IS NOT NULL
  )
  SELECT
    fleet.carrier,
    a.name AS airline,
    ROUND(AVG(2013 - fleet.year), 2) AS avg_plane_age,
    COUNT(*) AS num_aircraft
  FROM fleet
  JOIN airlines AS a
    ON fleet.carrier = a.carrier
  GROUP BY fleet.carrier, a.name
  ORDER BY avg_plane_age DESC
""")

print(result)

shape: (16, 4)
┌─────────┬────────────────────────┬───────────────┬──────────────┐
│ carrier ┆ airline                ┆ avg_plane_age ┆ num_aircraft │
│ ---     ┆ ---                    ┆ ---           ┆ ---          │
│ str     ┆ str                    ┆ f64           ┆ u32          │
╞═════════╪════════════════════════╪═══════════════╪══════════════╡
│ MQ      ┆ Envoy Air              ┆ 35.5          ┆ 4            │
│ AA      ┆ American Airlines Inc. ┆ 25.4          ┆ 164          │
│ DL      ┆ Delta Air Lines Inc.   ┆ 17.67         ┆ 609          │
│ UA      ┆ United Air Lines Inc.  ┆ 13.05         ┆ 589          │
│ WN      ┆ Southwest Airlines Co. ┆ 11.01         ┆ 569          │
│ …       ┆ …                      ┆ …             ┆ …            │
│ B6      ┆ JetBlue Airways        ┆ 6.5           ┆ 187          │
│ AS      ┆ Alaska Airlines Inc.   ┆ 5.16          ┆ 83           │
│ F9      ┆ Frontier Airlines Inc. ┆ 5.0           ┆ 23           │
│ VX      ┆ Virgin America       

### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [57]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
print(result)

# Now write your join query
result = ctx.execute("""
  SELECT
    f.year, f.month, f.day, f.dep_time,
    f.origin, f.dest, f.carrier, f.flight,
    f.dep_delay,
    w.wind_speed, w.precip
  FROM flights AS f
  JOIN weather AS w
    ON f.origin = w.origin
  AND f.time_hour = w.time_hour
  WHERE f.dep_delay > 30
    AND (w.wind_speed > 20 OR w.precip > 0.1)
  ORDER BY f.dep_delay DESC
  LIMIT 50
""")

print(result)

shape: (5, 15)
┌────────┬──────┬───────┬─────┬───┬────────┬──────────┬───────┬─────────────────────────┐
│ origin ┆ year ┆ month ┆ day ┆ … ┆ precip ┆ pressure ┆ visib ┆ time_hour               │
│ ---    ┆ ---  ┆ ---   ┆ --- ┆   ┆ ---    ┆ ---      ┆ ---   ┆ ---                     │
│ str    ┆ i64  ┆ i64   ┆ i64 ┆   ┆ f64    ┆ f64      ┆ f64   ┆ datetime[μs, UTC]       │
╞════════╪══════╪═══════╪═════╪═══╪════════╪══════════╪═══════╪═════════════════════════╡
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.0   ┆ 10.0  ┆ 2013-01-01 06:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.3   ┆ 10.0  ┆ 2013-01-01 07:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.5   ┆ 10.0  ┆ 2013-01-01 08:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.2   ┆ 10.0  ┆ 2013-01-01 09:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1011.9   ┆ 10.0  ┆ 2013-01-01 10:00:00 UTC │
└────────┴──────┴───────┴─────┴───┴────────┴──────────┴───────┴──────────────────────

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [58]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS num_flights
  FROM flights AS f
  JOIN planes  AS p
    ON f.tailnum = p.tailnum
  GROUP BY p.manufacturer, p.model
  ORDER BY num_flights DESC
  LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────────┐
│ manufacturer                  ┆ model           ┆ num_flights │
│ ---                           ┆ ---             ┆ ---         │
│ str                           ┆ str             ┆ u32         │
╞═══════════════════════════════╪═════════════════╪═════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278       │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027       │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716       │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553       │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051       │
│ BOEING                        ┆ 737-824         ┆ 13809       │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807       │
│ BOEING                        ┆ 737-7H4         ┆ 10389       │
│ BOEING                        ┆ 757-222         ┆ 9150        │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932     

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [59]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    f.origin,
    ao.origin_name,
    f.dest,
    ad.dest_name,
    COUNT(*) AS num_flights,
    ROUND(AVG(f.dep_delay), 2) AS avg_dep_delay,
    ROUND(AVG(CASE WHEN f.dep_delay > 30 THEN 1.0 ELSE 0.0 END) * 100, 2) AS pct_delayed_gt30
  FROM flights AS f
  JOIN (SELECT faa, name AS origin_name FROM airports) AS ao
    ON f.origin = ao.faa
  JOIN (SELECT faa, name AS dest_name FROM airports) AS ad
    ON f.dest = ad.faa
  GROUP BY f.origin, ao.origin_name, f.dest, ad.dest_name
  ORDER BY num_flights DESC
  LIMIT 10
""")

print(result)

shape: (10, 7)
┌────────┬─────────────────┬──────┬─────────────────┬─────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name     ┆ dest ┆ dest_name       ┆ num_flights ┆ avg_dep_delay ┆ pct_delayed_gt │
│ ---    ┆ ---             ┆ ---  ┆ ---             ┆ ---         ┆ ---           ┆ 30             │
│ str    ┆ str             ┆ str  ┆ str             ┆ u32         ┆ f64           ┆ ---            │
│        ┆                 ┆      ┆                 ┆             ┆               ┆ f64            │
╞════════╪═════════════════╪══════╪═════════════════╪═════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy  ┆ LAX  ┆ Los Angeles     ┆ 11262       ┆ 8.52          ┆ 9.83           │
│        ┆ Intl            ┆      ┆ Intl            ┆             ┆               ┆                │
│ LGA    ┆ La Guardia      ┆ ATL  ┆ Hartsfield      ┆ 10263       ┆ 11.45         ┆ 12.25          │
│        ┆                 ┆      ┆ Jackson Atlanta ┆             ┆         

In [60]:
# Write your SQL query here
result = ctx.execute("""
  SELECT
    f.origin,
    ao.origin_name,
    f.dest,
    ad.dest_name,
    COUNT(*) AS num_flights,
    ROUND(AVG(f.dep_delay), 2) AS avg_dep_delay,
    ROUND(AVG(CASE WHEN f.dep_delay > 30 THEN 1.0 ELSE 0.0 END) * 100, 2) AS pct_delayed_gt30
  FROM flights AS f
  JOIN (SELECT faa, name AS origin_name FROM airports) AS ao
    ON f.origin = ao.faa
  JOIN (SELECT faa, name AS dest_name FROM airports) AS ad
    ON f.dest = ad.faa
  GROUP BY f.origin, ao.origin_name, f.dest, ad.dest_name
  HAVING COUNT(*) >= 200
  ORDER BY pct_delayed_gt30 DESC
  LIMIT 10
""")

print(result)

shape: (10, 7)
┌────────┬─────────────────┬──────┬─────────────────┬─────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name     ┆ dest ┆ dest_name       ┆ num_flights ┆ avg_dep_delay ┆ pct_delayed_gt │
│ ---    ┆ ---             ┆ ---  ┆ ---             ┆ ---         ┆ ---           ┆ 30             │
│ str    ┆ str             ┆ str  ┆ str             ┆ u32         ┆ f64           ┆ ---            │
│        ┆                 ┆      ┆                 ┆             ┆               ┆ f64            │
╞════════╪═════════════════╪══════╪═════════════════╪═════════════╪═══════════════╪════════════════╡
│ LGA    ┆ La Guardia      ┆ SBN  ┆ South Bend Rgnl ┆ 6           ┆ 31.33         ┆ 50.0           │
│ EWR    ┆ Newark Liberty  ┆ TYS  ┆ Mc Ghee Tyson   ┆ 323         ┆ 41.82         ┆ 37.46          │
│        ┆ Intl            ┆      ┆                 ┆             ┆               ┆                │
│ EWR    ┆ Newark Liberty  ┆ TUL  ┆ Tulsa Intl      ┆ 315         ┆ 34.91   

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [61]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here

SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘
